In [ ]:
# Run this cell to install DiffeRT and its dependencies, e.g., on Google Colab

try:
    import differt  # noqa: F401
except ImportError:
    import sys  # noqa: F401

    !{sys.executable} -m pip install differt[all]

# Ray Tracing at City Scale

In [ ]:
import jax.numpy as jnp
from tqdm.notebook import trange

import differt.plotting as dplt
from differt.geometry import TriangleMesh
from differt.scene import TriangleScene

In [ ]:
mesh_file = "bruxelles.obj"
mesh = TriangleMesh.load_obj(mesh_file)

tx = jnp.array([-40.0, 75, 30.0])
rx = jnp.array([+20.0, 108.034, 1.50])

scene = TriangleScene(transmitters=tx, receivers=rx, mesh=mesh)

with dplt.reuse() as canvas:
    scene.plot()
    for order in trange(
        0,
        2,
        leave=False,
    ):  # You probably don't want to try order > 1 (too slow if testing all paths)
        # Prepare input arrays
        for paths in scene.compute_paths(order, chunk_size=2_000_000):
            paths.plot()

view = dplt.view_from_canvas(canvas)
view.camera.set_state(
    {
        "scale_factor": 138.81554751457762,
        "center": (20.0, 108.034, 46.0),
        "fov": 45.0,
        "elevation": 13.0,
        "azimuth": 39.0,
        "roll": 0.0,
    },
)

canvas

In [ ]:
# This tutorial is TODO